# Referential Relationship Analysis

In [ ]:
# ! py -m pip install thefuzz

In [1]:
import pandas as pd
# import numpy as np
# import thefuzz
from thefuzz import fuzz
from thefuzz import process
import glob

In [74]:
# TODO: Implement fuzzy matching
# TODO: Output results to same file
# TODO: Implement asset suite-style analysis by keyword

## Set search parameters
match_thresh = 90
search_btwn_apps = False
search_list = ['ID','CODE','NUM']

In [48]:
domain_path = 'C:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Training\\3. Data & Modeling'
domain = 'Training'
applications = {'TRAIN':domain_path+'\\Data Dictionary\\Working\\TRAIN_ColumnInfo_20240722.xlsx'}
module_sheets = {'TRAIN':{'Columns':'n/a'}} ## {APP NAME:{SHEET NAME:MODULE NAME}}

In [49]:
domain_gloss = pd.DataFrame()
dd_dict = {}
for app, filename in applications.items():
    dd_dict = pd.read_excel(filename, sheet_name = list(module_sheets[app].keys()))
    tables = pd.read_excel(filename, sheet_name = 'Tables')
    for module, df in dd_dict.items():
        df['Domain'] = domain
        df['Application'] = app
        df['Module'] = module_sheets[app][module]
        df = df.merge(tables, on='TABLENAME', how='left')
        domain_gloss = pd.concat([domain_gloss,df])

In [50]:
domain_gloss#[domain_gloss['COLNAME'].str.contains('|'.join(search_list))].sort_values('COLNAME')

,TABLENAME,COLNAME,Friendly Name,IncludeInView,Description,PK_ordinal_position,Referenced Table,Referenced Column,Reviewer Initials,Domain,Application,Module,Table Description
0,ADMINISTRATOR_ORG,EMP_SSN_NUM,Employee SNumber,Y,SNumber of the employee associated with the ad...,1.0,EMPLOYEE,EMP_SSN_NUM,NaN,Training,TRAIN,n/a,This table describes administrators and their ...
1,ADMINISTRATOR_ORG,CLASS_ADMIN,Class Admin,Y,Indicates if the user is a class administrator...,NaN,NaN,NaN,NaN,Training,TRAIN,n/a,This table describes administrators and their ...
2,ADMINISTRATOR_ORG,CONTRACT_ID,Contract ID,Y,Unique identifier for the contractor associate...,NaN,NaN,NaN,NaN,Training,TRAIN,n/a,This table describes administrators and their ...
3,ADMINISTRATOR_ORG,ORG_CODE,Organization Code,Y,Unique identifier for the organization associa...,NaN,ORG,ORG_CODE,NaN,Training,TRAIN,n/a,This table describes administrators and their ...
4,ADMINISTRATOR_ORG,ORG_ENROLLER,Organization Enroller,Y,Indicates if the user is has enrolling privileges,NaN,ORG,ORG_CODE,NaN,Training,TRAIN,n/a,This table describes administrators and their ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,TRAIN_MED_CERT_CODE,CERT_DESC,Cert Description,Y,Description of the medical certification,NaN,NaN,NaN,NaN,Training,TRAIN,n/a,Lookup table for cert_code field in train_medi...
379,TRAIN_MEDICAL_CERTS,CERT_CODE,Cert Code,Y,Unique identifier for the medical certificatio...,1.0,TRAIN_MED_CERT_CODE,CERT_CODE,NaN,Training,TRAIN,n/a,This table contains all records related to med...
380,TRAIN_MEDICAL_CERTS,EMP_SSN_NUM,Employee SNumber,Y,SNumber of the employee associated with the me...,2.0,EMPLOYEE,EMP_SSN_NUM,NaN,Training,TRAIN,n/a,This table contains all records related to med...
381,TRAIN_MEDICAL_CERTS,END_DATE,End Date,Y,Date when the medical certification expires,NaN,NaN,NaN,NaN,Training,TRAIN,n/a,This table contains all records related to med...


## Column Name Matching

In [51]:
domain_subgloss = domain_gloss[['Application','TABLENAME','COLNAME','Primary Key','Description','Table Description']]#''
domain_subgloss['idx'] = domain_subgloss['TABLENAME']+domain_subgloss['COLNAME']
domain_subgloss = domain_subgloss.set_index('idx')

C:\Users\mounsl\AppData\Local\Temp\1\ipykernel_696\1036352198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domain_subgloss['idx'] = domain_subgloss['TABLENAME']+domain_subgloss['COLNAME']


In [65]:
domain_subgloss[domain_subgloss['COLNAME']=='EMP_SSN_NUM']

,Application,TABLENAME,COLNAME,PK_ordinal_position,Description,Table Description
idx,,,,,,
ADMINISTRATOR_ORGEMP_SSN_NUM,TRAIN,ADMINISTRATOR_ORG,EMP_SSN_NUM,1.0,SNumber of the employee associated with the ad...,This table describes administrators and their ...
ATTEND_TESTEMP_SSN_NUM,TRAIN,ATTEND_TEST,EMP_SSN_NUM,2.0,SNumber of the employee who took the test,Contains records with basic information about ...
CLASS_ATTENDEMP_SSN_NUM,TRAIN,CLASS_ATTEND,EMP_SSN_NUM,2.0,SNumber of the employee who attended the class,This table includes the Learners that are part...
COURSE_WAITING_LISTEMP_SSN_NUM,TRAIN,COURSE_WAITING_LIST,EMP_SSN_NUM,2.0,SNumber of the employee who is on the waiting ...,This table is where administrators and trainin...
EMP_ADMIN_DOCEMP_SSN_NUM,TRAIN,EMP_ADMIN_DOC,EMP_SSN_NUM,3.0,SNumber of the employee associated with the ad...,This table contains records for all admin form...
EMP_JOBEMP_SSN_NUM,TRAIN,EMP_JOB,EMP_SSN_NUM,1.0,SNumber of the employee associated with a job,This is a bridge table that assigns job codes ...
EMP_QLFEMP_SSN_NUM,TRAIN,EMP_QLF,EMP_SSN_NUM,NaN,SNumber of the employee associated with the qu...,This table shows all instances of a qualificat...
EMP_READINGEMP_SSN_NUM,TRAIN,EMP_READING,EMP_SSN_NUM,1.0,SNumber of the employee associated with the re...,This table contains records for all the readin...
EMPLOYEEEMP_SSN_NUM,TRAIN,EMPLOYEE,EMP_SSN_NUM,1.0,SNumber of an employee,This table includes data about all employees a...


In [66]:
col_names = list(domain_gloss['COLNAME'].unique())

In [67]:
col_names

['EMP_SSN_NUM',
 'CLASS_ADMIN',
 'CONTRACT_ID',
 'ORG_CODE',
 'ORG_ENROLLER',
 'PJC_ACCESS',
 'PRIMARY_TC',
 'REPORTS',
 'SECONDARY_TC',
 'TP_ACCESS',
 'UPDATE_DATE',
 'UPDATE_UID',
 'CLASS_CODE',
 'TEST_DATE',
 'COURSE_CODE',
 'COURSE_REV_NUM',
 'EMP_NAME',
 'EXAM_TYPE_CODE',
 'INSTR_TYPE_CODE',
 'PASS_FAIL_FLAG',
 'TEST_CODE',
 'TEST_GRADE_PCT',
 'TEST_ROSTER_NUM',
 'CHANGE_HISTORY',
 'CLASS_COMMENT',
 'CLASS_FEE',
 'CLASS_TITLE',
 'CLS_SIZE_MAX_QTY',
 'CLS_SIZE_MIN_QTY',
 'DURATION_HRS',
 'ENROLLMENT_TYPE_CODE',
 'EVENT_CLASSIF_TEXT',
 'INSTR_FLAG',
 'INSTR_ORG_CODE',
 'INSTR_VEND_DESC',
 'LOCATION',
 'PILOT_CLASS_FLAG',
 'POC',
 'PREVIOUS_ENROLLMENT_TYPE_CODE',
 'TECH_TRNG_FLAG',
 'VEND_CODE',
 'VIRTUAL_CLASS_FLAG',
 'VIRTUAL_LINK',
 'ENROLLMENT_DATE',
 'ACTIVITY_CODE',
 'CHARGEBACK_FEED_DT',
 'CLASS_ATTEND_STATUS_CODE',
 'CLASS_CHRG_FLAG',
 'CLASS_ROSTER_NUM',
 'CLASS_TEST_FLAG',
 'COMMENTS',
 'COMPLETE_DATE',
 'EMP_CLASS_FEE',
 'ENROLLER_SSN',
 'RETRAIN_DATE',
 'SEND_NOTE',
 'TRA

In [75]:
match_df = pd.DataFrame(columns=['Search Term','Matched Column Name','Matched Table','Primary Key','Match Ratio','Match Application'])
for col in col_names:
    matches_list = []
    matches = process.extract(col, domain_subgloss['COLNAME'], scorer=fuzz.ratio, limit=None) # list of tuples [(matched_colname, match_ratio, index of match)]
    for result in matches:
        match_colname = result[0]
        match_ratio = result[1]
        match_key = result[2]
        if match_ratio >= match_thresh:
            # if app == app:
            matches_list.append({ # replaces previous entries
                        'Search Term':col,
                        'Matched Column Name':match_colname,
                        'Match Ratio':result[1],
                        'Match Application':domain_subgloss.loc[match_key]['Application'],
                        'Matched Table':domain_subgloss.loc[match_key]['TABLENAME'],
                        'Primary Key': domain_subgloss.loc[match_key]['PK_ordinal_position']})
    if len(matches_list)>1:
        new_matches_df = pd.DataFrame(matches_list)
        match_df = pd.concat([match_df,new_matches_df])


C:\Users\mounsl\AppData\Local\Temp\1\ipykernel_696\135764327.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  match_df = pd.concat([match_df,new_matches_df])


In [76]:
match_df.to_excel('TRAIN_possibleReferentialRelationships.xlsx', index=False)

## Substring Analysis

In [34]:
substrings = domain_gloss[domain_gloss['COLNAME'].str.contains('REF|NUM|NBR', case=False)]

In [54]:
substrings.to_excel('Asset Suite Reference Keys.xlsx', index=False, columns=['COLNAME','TABLENAME','PK_ordinal_position','Definition','Table Description'])